In [1]:
import numpy as np
from collections import deque, Counter

## Part 1

In [2]:
def read_text_file_as_grid(file_path):
    with open(file_path, 'r') as file:
        grid = [list(line.strip()) for line in file if line.strip()]
    return np.array(grid)

In [3]:
def is_inside_grid(grid, x, y):
    rows, cols = grid.shape
    return 0 <= x < rows and 0 <= y < cols

def is_valid_move(grid, x, y):
    return is_inside_grid(grid, x, y) and grid[x][y] != '#'

def find_start_and_end(grid):
    start = tuple(np.argwhere(grid=="S")[0])
    end = tuple(np.argwhere(grid=="E")[0])
    return start, end

def bfs(grid):
    start, end = find_start_and_end(grid)
    start_x, start_y = start
    goal_x, goal_y = end
    
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    queue = deque([(start_x, start_y, [])])
    
    visited = set()
    visited.add((start_x, start_y))
    
    while queue:
        x, y, path = queue.popleft()
        
        if (x, y) == (goal_x, goal_y):
            return path + [(x, y)]
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(grid, nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny, path + [(x, y)]))
    
    return None

def bfs_with_child_tracking(grid):
    start, end = find_start_and_end(grid)
    start_x, start_y = start
    goal_x, goal_y = end
    
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    queue = deque([(start_x, start_y, 0)])
    
    visited = set()
    visited.add((start_x, start_y))
    
    child_nodes = {}
    
    while queue:
        x, y, path_length = queue.popleft()
        
        if (x, y) == (goal_x, goal_y):
            return path_length, child_nodes
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(grid, nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny, path_length + 1))
                child_nodes[(x, y)] = (nx, ny)
    
    return None, None

In [4]:
def test_block_removal(grid, min_steps_saved=1):
    base_length, _ = bfs_with_child_tracking(grid)
    
    wall_indeces = np.argwhere(grid == "#")
    
    cheats = {}

    for x,y in wall_indeces:
        grid[x][y] = "."
        
        length, child_nodes = bfs_with_child_tracking(grid)
        
        if length is not None and length < base_length:
            saved_steps = base_length - length
            
            if (child_nodes and (x, y) in child_nodes) and saved_steps >= min_steps_saved:
                best_next_step = child_nodes[(x, y)]
                cheats[(best_next_step, (x, y))] = saved_steps

        grid[x][y] = "#"

    return cheats


In [5]:
input_file = "example1.txt"
grid = read_text_file_as_grid(input_file)

In [6]:
shortest_path = bfs(grid)
base_length = len(shortest_path)-1
print(len(shortest_path)-1)

84


In [7]:
cheats = test_block_removal(grid)
cheat_counter = Counter(cheats.values())
print(sorted(cheat_counter.items()))

[(2, 14), (4, 14), (6, 2), (8, 4), (10, 2), (12, 3), (20, 1), (36, 1), (38, 1), (40, 1), (64, 1)]


In [52]:
input_file = "input.txt"
grid = read_text_file_as_grid(input_file)
cheats = test_block_removal(grid, min_steps_saved=100)
cheat_counter = Counter(cheats.values())
count_cheat_paths = sum(cheat_counter.values())
print(f"final count of cheats: {count_cheat_paths}")

final count of cheats: 1363


## Part 2

In [69]:
from itertools import combinations

def find_nearby_walls(grid, max_distance=19):
    start, _ = find_start_and_end(grid)
    wall_indices = np.argwhere(grid == "#")
    
    nearby_walls = []
    
    for wx, wy in wall_indices:
        queue = deque([(start[0], start[1], 0)]) 
        visited = set()
        visited.add(start)
        
        while queue:
            x, y, dist = queue.popleft()
            
            if dist > max_distance:
                break
            
            if (x, y) == (wx, wy):
                nearby_walls.append((wx, wy, dist))
                break
            
            for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                nx, ny = x + dx, y + dy
                if is_inside_grid(grid, nx, ny) and (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append((nx, ny, dist + 1))
    
    return nearby_walls

def generate_removal_scenarios(nearby_walls):
    all_scenarios = []
    for r in range(1, len(nearby_walls) + 1):
        for removal_combo in combinations(nearby_walls, r):
            all_scenarios.append(removal_combo)
    
    return all_scenarios

def test_block_removal(grid, min_steps_saved=1):
    base_length, _ = bfs_with_child_tracking(grid)
    
    wall_indices = np.argwhere(grid == "#")
    
    cheats = {}
    nearby_walls = find_nearby_walls(grid, max_distance=19)
    
    removal_scenarios = generate_removal_scenarios(nearby_walls)
    
    for removal_set in removal_scenarios:
        original_grid = grid.copy()
        
        for wx, wy, _ in removal_set:
            grid[wx][wy] = "."
        
        length, child_nodes = bfs_with_child_tracking(grid)
        
        if length is not None and length < base_length:
            saved_steps = base_length - length

            last_replacement = removal_set[-1] 
            if (child_nodes and (last_replacement[0], last_replacement[1]) in child_nodes) and saved_steps >= min_steps_saved:
                best_next_step = child_nodes[(last_replacement[0], last_replacement[1])]
                cheats[(best_next_step, (last_replacement[0], last_replacement[1]))] = saved_steps
        
        grid = original_grid.copy()
    
    return cheats